# **PROJECT2 : LA Airbnb Listing**


## **ETL : Extract, Transform, Load Project**

* <b>Extract:</b>   read the data, often from mupltiple sources/formats.<br>
* <b>Transform:</b>  clean and structure the data to suit business needs.<br>
* <b>Load:</b>  load the data into a database for storage that can be used for future analysis or business use.

### Data source:
* https://www.kaggle.com/oindrilasen/la-airbnb-listings


 <b>DATA EXTRACTION</b><br>
 
   <b>STEP 1: Import Modules And Data</b>

In [60]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os

In [109]:
#Read the csv file
path = os.path.join("Resources", "Detail_listings.csv")
df=pd.read_csv(path)
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (87) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,16228948,https://www.airbnb.com/rooms/16228948,20170502172350,2017-05-03,Robert Ranch in Los Angeles for filming/wedding,Stunning Ranch Estate situated on 20 acres and...,NaN,Stunning Ranch Estate situated on 20 acres and...,none,NaN,...,NaN,f,NaN,NaN,t,strict,f,f,1,NaN
1,8909380,https://www.airbnb.com/rooms/8909380,20170502172350,2017-05-03,Nice quiet private ranch house,NaN,Very private and quite . Hiking horse riding a...,Very private and quite . Hiking horse riding a...,none,I'm in the country what's not to love about that,...,9.0,f,NaN,NaN,f,flexible,f,f,1,1.91
2,14078522,https://www.airbnb.com/rooms/14078522,20170502172350,2017-05-03,Room w/private bath on Small Ranch,My place is close to Six Flags Magic Mountain ...,Room with queen bed and private bath,My place is close to Six Flags Magic Mountain ...,none,Horse area. Trails for hiking and riding. Quie...,...,10.0,f,NaN,NaN,f,flexible,f,f,1,1.72
3,13006928,https://www.airbnb.com/rooms/13006928,20170502172350,2017-05-03,Miller Ranch Bunkhouse,"Our place is close to great views, equestrian ...",Please know that we live on a dirt road but it...,"Our place is close to great views, equestrian ...",none,NaN,...,10.0,f,NaN,NaN,t,flexible,f,f,1,2.12
4,7898757,https://www.airbnb.com/rooms/7898757,20170502172350,2017-05-03,Studio with full bathroom,Fully furnished studio only a 10 min drive fro...,NaN,Fully furnished studio only a 10 min drive fro...,none,NaN,...,10.0,f,NaN,"City of Los Angeles, CA",f,flexible,f,f,1,0.18


<b>STEP2. DATA EXPLORATION</b><br>
Explore Datasets to list Columns and their respective Data-types

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31253 entries, 0 to 31252
Data columns (total 95 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                31253 non-null  int64  
 1   listing_url                       31253 non-null  object 
 2   scrape_id                         31253 non-null  int64  
 3   last_scraped                      31253 non-null  object 
 4   name                              31246 non-null  object 
 5   summary                           30417 non-null  object 
 6   space                             21866 non-null  object 
 7   description                       31238 non-null  object 
 8   experiences_offered               31253 non-null  object 
 9   neighborhood_overview             18575 non-null  object 
 10  notes                             13815 non-null  object 
 11  transit                           18164 non-null  object 
 12  acce

<b>STEP3: DATA CLEANING<b>

In [63]:
# Ensure each listing is a unique property
# len(df.listing_url.unique())
len(df)

31253

In [64]:
def price_cleaner(df, column):
    '''
    cleans str price by removing dollar sign, commas, and changing dtype to float
    
    df : dataframe to change
    column: selected column in quotes (eg. 'cost_column')
    '''
    df[column] = df[column].str.replace(',', '')
    df[column] = df[column].str.replace('$', '')
    df[column] = df[column].astype(float)

In [77]:
#Establish baseline price column for comparisons
listing_df = df[["id", "listing_url", "name", "price"]].copy().reset_index(drop=True)

#Change price to float (not str)
price_cleaner(listing_df, "price")

listing_df.dropna(inplace=True)

listing_df

,id,listing_url,name,price
0,16228948,https://www.airbnb.com/rooms/16228948,Robert Ranch in Los Angeles for filming/wedding,3000.0
1,8909380,https://www.airbnb.com/rooms/8909380,Nice quiet private ranch house,50.0
2,14078522,https://www.airbnb.com/rooms/14078522,Room w/private bath on Small Ranch,55.0
3,13006928,https://www.airbnb.com/rooms/13006928,Miller Ranch Bunkhouse,150.0
4,7898757,https://www.airbnb.com/rooms/7898757,Studio with full bathroom,30.0
...,...,...,...,...
31248,18445825,https://www.airbnb.com/rooms/18445825,"Gorgeous Huge bedroom with Extra,s",75.0
31249,18192212,https://www.airbnb.com/rooms/18192212,Ocean View Malibu Dream - April Deal $205/night,205.0
31250,17339165,https://www.airbnb.com/rooms/17339165,"Beautifull Villa to Pacific Coastin, Malibu.",500.0
31251,17827299,https://www.airbnb.com/rooms/17827299,Entire Lower Level Portion of home,65.0


In [102]:
#neighbourhood dataframe
neighborhood_df=df[["id","neighbourhood_cleansed"]].dropna().reset_index(drop=True).reset_index()
neighborhood_df = neighborhood_df.rename(columns={"id":"listing_id", "index":"id"})
neighborhood_df.dropna(inplace=True)
neighborhood_df

,id,listing_id,neighbourhood_cleansed
0,0,16228948,Acton
1,1,8909380,Acton
2,2,14078522,Acton
3,3,13006928,Acton
4,4,7898757,Adams-Normandie
...,...,...,...
31248,31248,18445825,Whittier
31249,31249,18192212,Malibu
31250,31250,17339165,Malibu
31251,31251,17827299,La Habra Heights


In [92]:
# host info data frame
host_df = df[[
    "id", "host_id","host_url", "host_name", "host_since", "host_is_superhost", "host_total_listings_count"
]].dropna().reset_index(drop=True).reset_index()

host_df.dropna(inplace=True)
host_df = host_df.rename(columns={"host_is_superhost":"superhost_status","id":"listing_id","index":"id"})
host_df.head()

,id,listing_id,host_id,host_url,host_name,host_since,superhost_status,host_total_listings_count
0,0,16228948,54345288,https://www.airbnb.com/users/show/54345288,Robert,2016-01-12,f,1.0
1,1,8909380,44800067,https://www.airbnb.com/users/show/44800067,Kevin,2015-09-22,t,1.0
2,2,14078522,84615808,https://www.airbnb.com/users/show/84615808,Kirsten,2016-07-19,t,1.0
3,3,13006928,71743973,https://www.airbnb.com/users/show/71743973,Beverly,2016-05-12,t,1.0
4,4,7898757,41646908,https://www.airbnb.com/users/show/41646908,Ian,2015-08-16,f,1.0


In [100]:
# Property stats dataframe
property_stats_df = df[[
    "id","property_type", "room_type", "bedrooms","bathrooms"
]].dropna().reset_index(drop=True).reset_index()
#Change column names
property_stats_df = property_stats_df.rename(columns={"id":"listing_id", "index":"id"})
property_stats_df

,id,listing_id,property_type,room_type,bedrooms,bathrooms
0,0,16228948,House,Entire home/apt,5.0,7.0
1,1,8909380,House,Private room,1.0,1.0
2,2,14078522,House,Private room,1.0,1.0
3,3,13006928,Other,Entire home/apt,1.0,1.0
4,4,7898757,Apartment,Private room,1.0,1.0
...,...,...,...,...,...,...
31119,31119,18445825,House,Private room,1.0,1.0
31120,31120,18192212,Apartment,Private room,1.0,1.0
31121,31121,17339165,Villa,Entire home/apt,5.0,5.0
31122,31122,17827299,House,Private room,2.0,1.0


In [69]:
# Associated Fees dataframe
fees_df = df[[
    "id","security_deposit", "cleaning_fee","extra_people"
]].reset_index(drop=True)

price_cleaner(fees_df, "security_deposit")
price_cleaner(fees_df, "cleaning_fee")
price_cleaner(fees_df, "extra_people")

fees_df["security_deposit"]
fees_df["cleaning_fee"]
fees_df["extra_people"]

fees_df = fees_df.rename(columns={"id":"listing_id"})

fees_df = fees_df.fillna(0)
fees_df.head()

,listing_id,security_deposit,cleaning_fee,extra_people
0,16228948,2000.0,200.0,0.0
1,8909380,0.0,0.0,0.0
2,14078522,0.0,0.0,10.0
3,13006928,0.0,35.0,0.0
4,7898757,0.0,5.0,10.0


In [70]:
# Property Reviews Dataframe
reviews_df = df[[
    "id","review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness","review_scores_location","review_scores_value","reviews_per_month"
]].dropna().reset_index(drop=True)
#Change column names
reviews_df = reviews_df.rename(columns={"id":"listing_id"})

reviews_df.columns = reviews_df.columns.str.replace('review_scores_', '')
reviews_df

,listing_id,rating,accuracy,cleanliness,location,value,reviews_per_month
0,8909380,93.0,10.0,10.0,9.0,9.0,1.91
1,14078522,100.0,10.0,10.0,10.0,10.0,1.72
2,13006928,100.0,10.0,10.0,10.0,10.0,2.12
3,7898757,93.0,10.0,9.0,9.0,10.0,0.18
4,16068508,98.0,10.0,10.0,10.0,10.0,7.92
...,...,...,...,...,...,...,...
23629,15860751,70.0,7.0,7.0,10.0,8.0,0.45
23630,1312336,91.0,9.0,9.0,9.0,9.0,3.34
23631,17302114,100.0,10.0,9.0,10.0,10.0,2.25
23632,16027372,96.0,9.0,9.0,10.0,10.0,3.45


<b>STEP4: DATA LOADING<b>

In [72]:
#Connecting to Postegres DB
rds_connection_string = "postgres:postgres@localhost:5432/airbnb_la_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [108]:
#Checking table names in DB
engine.table_names()

['listing', 'neighborhood', 'host', 'property_stats', 'fees', 'reviews']

<b>Importing dataframes into Postgres DB <b>

In [78]:
listing_df.to_sql(name='listing', con=engine, if_exists='append', index=False)

In [103]:
neighborhood_df.to_sql(name='neighborhood', con=engine, if_exists='append', index=False)

In [104]:
host_df.to_sql(name='host', con=engine, if_exists='append', index=False)

In [105]:
property_stats_df.to_sql(name='property_stats', con=engine, if_exists='append', index=False)

In [106]:
fees_df.to_sql(name='fees', con=engine, if_exists='append', index=False)

In [107]:
reviews_df.to_sql(name='reviews', con=engine, if_exists='append', index=False)